In [17]:

print(49*48)
print(2.352/2.4)
print(int('0xBF68',0))

2352
0.98
49000


In [1]:
# Toets basiese skryf na die seriepoort
import serial
ser = serial.Serial('/dev/ttyUSB0')  # open serial port
print(ser.name)         # check which port was really used
#ser.write(b'hello')     # write a string
ser.close()             # close port

/dev/ttyUSB0


In [2]:
# Probeer 1200 baud en 115200
with serial.Serial('/dev/ttyUSB0', 1200, timeout=1) as ser:
    x = ser.read()          # read one byte
    s = ser.read(10)        # read up to ten bytes (timeout)
    #line = ser.readline()   # read a '\n' terminated line
    
print(s)

b''


In [4]:
from binascii import unhexlify
string='0281E1B1'
print(unhexlify(string))

b'\x02\x81\xe1\xb1'


In [77]:
# Deel die heks uitstring op in die dele wat die data bevat soos 
# die hoeveelheid energie oor in battery

import struct

def wysHeksString(insetstring):
    print('Heksadesimale ASCII waarde')
    print(insetstring)
    print('Heksadesimale string')
    getalinheks = bytearray.fromhex(insetstring).decode()
    print(getalinheks)
    print('Desimale string')
    print(int(getalinheks, 16))
    
    return 0

def uitsetGetalHeksString(insetstring):
    # Neem die insetstring van die battery ontvang en kry die ASCII waarde daarvan
    getalinheks = bytearray.fromhex(insetstring).decode()
    # getalinheks word nou omgeskakel na 'n desimale waarde wat gebruik kan word.
    uitsetgetal = int(getalinheks, 16)
    
    return uitsetgetal

def uitsetGetalHeksStringSignInt(insetstring):
    # Neem die insetstring van die battery ontvang en kry die ASCII waarde daarvan
    getalinheks = bytearray.fromhex(insetstring).decode()
    # getalinheks word nou omgeskakel na 'n desimale waarde wat gebruik kan word.
    uitsetgetal = int(getalinheks, 16)
    
    return uitsetgetal


def skryfLogLynBattery(leesvanbattery):
    a = leesvanbattery
    
    #Stroom in milliAmpere
    stroom = uitsetGetalHeksString(a[-78:-70])
    #Spanning in milliVolt van hele battery
    spanning = uitsetGetalHeksString(a[-70:-62])
    #Oorblywende energie in battery
    energieOor = uitsetGetalHeksString(a[-62:-54])
    # Totale energie in battery
    energietotaal = uitsetGetalHeksString(a[-50:-42])
    # Siklusse
    siklusse = uitsetGetalHeksString(a[-42:-34])
        
    uitsetloglyn = str(stroom) + ',' + str(spanning) + ',' + str(energieOor) + ',' + str(energietotaal) + ',' + str(siklusse)
    
    return uitsetloglyn

In [84]:
from binascii import unhexlify

# Opdrag vanaf paragraaf 5 in seriepoort handleiding
bytestosend = '7E3230303134363432453030323031464433350D'

# Probeer 1200 baud en 115200
with serial.Serial('/dev/ttyUSB0', 1200, timeout=5.0) as ser:
    x = ser.write(unhexlify(bytestosend)) # Stuur opdrag na die battery
    
    uitstring = ser.read(1000)

# Druk die greepstring (bytestring)
print('Greepstring')
print(uitstring)

# Skakel die greepstring om na heksadesimale string
print('Heksadesimale string')
a = uitstring.hex()
print(a)    

print(skryfLogLynBattery(a))

Greepstring
b'~20014600F07A11010F0CEF0CF10CF10CF10CF10CF10CF00CF10CEE0CEF0CF00CF00CF10CF10CF0050BE10BC30BC30BC30BB9FFF6C214FFFF04FFFF0010011864012110E1DD\r'
Heksadesimale string
7e3230303134363030463037413131303130463043454630434631304346313043463130434631304346313043463030434631304345453043454630434630304346303043463130434631304346303035304245313042433330424333304243333042423946464636433231344646464630344646464630303130303131383634303132313130453144440d
65526,49684,65535,65535,16


In [69]:
print('Aantal siklusse')
#31313030313231
#31304532
wysHeksString(a[-14:-6])
print('Totale energie')
wysHeksString(a[-18:-10])

#getalinheks = bytearray.fromhex("42463638").decode()
#print(getalinheks)
#print(int(getalinheks, 16))

Aantal siklusse
Heksadesimale ASCII waarde
31304532
Heksadesimale string
10E2
Desimale string
4322
Totale energie
Heksadesimale ASCII waarde
32313130
Heksadesimale string
2110
Desimale string
8464


0

# Voorbeeld uit handleiding

`7E` - Begin inligting
`32 30` - Weergawe 
`30 31` - Adres 
`34 36` - Kode vir litium battery
`30 30` - RTN Return kode 30 30 is 00 is alles het vlot verloop 
`43 30 36 45` - Lengte of LEN is die lengte van die data (met CHKSUM ingesluit) 
`31 31 30 31` - INFOFLAG

Data begin
`30 46` - Aantal selle
30 44 34 35 - Sel 1 Volts
30 44 34 34 
30 44 34 35 
30 44 34 34 30 44 34 35 30 44 34
34 30 44 33 45 30 44 34 35 30 44 34 41 30 44 34 41 30 44 34 42 30 44 34
41 30 44 34 41 30 44 34 41 30 44 34 41 30 35 30 42 43 33 30 42 43 33 30
42 43 33 30 42 43 44 30 42 43 44 30 30 30 30 43 37 32 35 42 46 36 38 30
32 
`43 33 35 30` - Totale energie (mAh) 
`30 30 30 32` - Aantal siklusse
`45 35 35 33` - Toetssom of CHKSUM
`0D` - Einde van inligting

# Voorbeeld uitset
`7e` - Begin inligting  
`32 30` - Weergawe  
`30 31` - Adres  
`34 36` - Kode vir litium battery  
`30 30` - RTN Return kode 30 30 is 00 is alles het vlot verloop  
`46 30 37 41` - Lengte  
`31 31 30 31` - INFOFLAG  

Data begin  
`30 46` - Aantal selle (15)  
`30 44 45 32` - Sel 1 Volts  
`30444532` - Sel 2 Volts  
`30444430` - Sel 3 Volts  
`30444533` - Sel 4 Volts  
`30444533` - Sel 5 Volts  
`30444534` - Sel 6 Volts  
`30444532` - Sel 7 Volts  
`30444531` - Sel 8 Volts  
`30444538` - Sel 9 Volts  
`30444537` - Sel 10 Volts  
`30444444` - Sel 11 Volts  
`30444537` - Sel 12 Volts  
`30444538` - Sel 13 Volts  
`30444537` - Sel 14 Volts  
`30444537` - Sel 15 Volts   
`30 35` - Aantal Temperatuur meetpunte  
`30424542` - Temperatuur 1  
`30424333` - Temperatuur 2  
`30424333` - Temperatuur 3  
`30424333` - Temperatuur 4  
`30424333` - Temperatuur 5 in Kelvin maal 10, dus is hierdie waarde van 3011 -> 301.1 Kelvin oftewel 28.1 grade Celcius.  

Stroom en spanning data:  
`30303030` - Stroom in milliAmpere  
`44303441` - Spanning in milliVolt van hele battery (53322mV of 53.3V)  
`46464646` - Oorblywende energie in battery (65535mAh).  Hierdie battery is 100% vol of SOC = 100% as jy hierdie waarde deur die totale energie in battery deel.  
`3034` - "User-defined" waarde.  In hierdie geval 4  
`46464646` - Totale energie in battery (65535mAh)  
`30303130` - Aantal siklusse van battery.  Die battery het al 16 volle siklusse gedoen.  

`303132313130303132313130` - Ekstra info  
`45323130` - Checksum  
`0d` - Einde van inligting  

In [70]:
wysHeksString('30303130')
#Stroom in milliAmpere
print(a[-78:-70])
#Spanning in milliVolt van hele battery
print(a[-70:-62])
#Oorblywende energie in battery
print(a[-62:-54])
# Totale energie in battery
print(a[-50:-42])
# Siklusse
print(a[-42:-34])

print(skryfLogLynBattery(a))

Heksadesimale ASCII waarde
30303130
Heksadesimale string
0010
Desimale string
16
30303030
44303441
46464646
46464646
30303130
0,53322,65535,65535,16


In [86]:
print(a[-78:-70])
getalinheks = bytearray.fromhex(a[-78:-70]).decode()
print(getalinheks)
h = '0000FFF6'
struct.unpack('>i', bytes.fromhex(h))

46464636
FFF6


(65526,)